In [25]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
import re
import string
from tqdm import tqdm
import nltk
from nltk.tokenize import word_tokenize
from textblob import TextBlob
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer


ImportError: The 'enchant' C library was not found and maybe needs to be installed.
See  https://pyenchant.github.io/pyenchant/install.html
for details


In [2]:
tweets = (open("./../Dataset/train/us_train.text", "r").readlines())

In [3]:
allTweets = pd.DataFrame(tweets, columns=['tweets'])


In [4]:
stopword = nltk.corpus.stopwords.words('english')
stopword += ['yr', 'year', 'woman', 'man', 'girl','boy','one', 'two', 'sixteen', 'yearold', 'fu', 'weeks', 'week',
        'treatment', 'associated', 'patients', 'may','day', 'case','old']
ps = PorterStemmer()


In [16]:
def clean_text(text):
    text = re.sub(r'@user','',text)
    text  = "".join([char.lower() for char in text if char not in string.punctuation])
    text  = "".join([char.lower() for char in text if char not in ['…','\n']])
    text = re.sub('[0-9]+', '', text)
    text = ''.join(i for i in text if ord(i)<128)
    text = re.sub('\s+', ' ', text)
    text = word_tokenize(text)    
    for i in range(len(text)):
        word = text[i]
        newword = ""
        prev = word[0]
        for j in range(1,len(word)):
            if word[j] == prev:
                pass
            else:
                newword += word[j]
                prev = word[j]
        text[i] = newword

    text = [word for word in text if word not in stopword]
    #text = [str(TextBlob(word).correct()) for word in text]
    text = [word for word in text if word not in stopword]
    text = [ps.stem(word) for word in text]
    text = [word for word in text if 10>len(word)>2]
    for word in text:
        if len(word)<=2:
            print(word)
    return text

In [17]:
vectorizer = CountVectorizer(analyzer=clean_text)
countVector = vectorizer.fit_transform(allTweets['tweets'][:100000])
print('{} Number of tweets has {} words'.format(countVector.shape[0], countVector.shape[1]))

100000 Number of tweets has 49862 words


In [18]:
countVector = countVector.astype(np.int8)

In [19]:
from sklearn import svm
from sklearn.model_selection import train_test_split

In [22]:
count_vect_df = pd.DataFrame(countVector.toarray(), columns=vectorizer.get_feature_names())
count_vect_df.head()



/home/karan/.local/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


KeyboardInterrupt: 

In [80]:
CountVectorizer?

Init signature:
CountVectorizer(
    *,
    input='content',
    encoding='utf-8',
    decode_error='strict',
    strip_accents=None,
    lowercase=True,
    preprocessor=None,
    tokenizer=None,
    stop_words=None,
    token_pattern='(?u)\\b\\w\\w+\\b',
    ngram_range=(1, 1),
    analyzer='word',
    max_df=1.0,
    min_df=1,
    max_features=None,
    vocabulary=None,
    binary=False,
    dtype=<class 'numpy.int64'>,
)
Docstring:     
Convert a collection of text documents to a matrix of token counts.

This implementation produces a sparse representation of the counts using
scipy.sparse.csr_matrix.

If you do not provide an a-priori dictionary and you do not use an analyzer
that does some kind of feature selection then the number of features will
be equal to the vocabulary size found by analyzing the data.

Read more in the :ref:`User Guide <text_feature_extraction>`.

Parameters
----------
input : {'filename', 'file', 'content'}, default='content'
    - If `'filename'`, the sequ